In [ ]:
import numpy as np
import pandas as pd
from category_encoders import TargetEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer

CAT_FEATURES = [
    "area_type",
    "city",
    "furnishing_status",
    "tenant_preferred",
    "point_of_contact",
]


def extract_floor(floor_info: str) -> int:
    """층수 컬럼에서 실제 층수만 추출합니다.

    현재 층수 정보는 'X out of Y'로 되어 있습니다.
    여기에서 X만 추출하여 정수로 반환합니다.
    Upper basement, Lower basement, Ground out 등은 모두 0층으로 변환합니다.

    Args:
        floor_info (str): 층수 정보
    """
    split_floor_info = floor_info.split()
    floor_str = split_floor_info[0]

    return int(floor_str) if floor_str.isnumeric() else 0


# extract_floor()가 str -> int라서, 이걸 감싸는 Wrapping function임


def floor_extractor(df: pd.DataFrame, col: str) -> pd.DataFrame:
    """`extract_floor()` 함수를 `FunctionTransformer`에 사용하기 위한
    Wrapping function입니다.

    Args:
        df (pd.DataFrame): 데이터프레임
        col (str): `extract_floor()`를 적용할 컬럼명
            `Floor`만 사용해야 함

    Returns:
        pd.DataFrame: 컬럼 처리 후 데이터
    """
    df[col] = df[col].apply(lambda x: extract_floor(x))
    return df


# 전처리 파이프라인 작성
# 1. 방의 크기는 제곱근을 적용함 (FunctionTransformer 사용)
# 2. 층수는 실제 층수를 추출하되 숫자가 아닌 Basement 등은 0층으로 표기함
# 3. 범주형 변수(CAT_FEATURES)는 타겟 인코딩 적용 (from category_encoders import TargetEncoder)
# ColumnTransform 으로 만드는 객체에는 Transform이 무조건 존재함. fit은 있을 수도 있고, 없을 수도 있음

preprocess_pipeline = ColumnTransformer(
    transformers=[
        ("sqrt_transformer", FunctionTransformer(np.sqrt), ["size"]),
        (
            "floor_extractor",
            FunctionTransformer(floor_extractor, kw_args={"col": "floor"}),
            ["floor"],
        ),
        ("target_encoder", TargetEncoder(cols=CAT_FEATURES), CAT_FEATURES),
    ],
    remainder="passthrough",
    verbose_feature_names_out=False,
)
preprocess_pipeline.set_output(transform="pandas")


In [1]:
import numpy as np
import pandas as pd
from category_encoders import TargetEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer

In [6]:
import os
from src.common.constants import ARTIFACT_PATH, DATA_PATH, LOG_FILEPATH
train_df = pd.read_csv(os.path.join(DATA_PATH, "bike_sharing_train.csv"))

In [9]:
train_df['datetime'] = pd.to_datetime(train_df['datetime'])

train_df['year'] = train_df['datetime'].dt.year
train_df['month'] = train_df['datetime'].dt.month
train_df['hour'] = train_df['datetime'].dt.hour
train_df['weekday'] = train_df['datetime'].dt.weekday

In [11]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5422 entries, 0 to 5421
Data columns (total 14 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   datetime    5422 non-null   datetime64[ns]
 1   season      5422 non-null   int64         
 2   holiday     5422 non-null   int64         
 3   workingday  5422 non-null   int64         
 4   weather     5422 non-null   int64         
 5   temp        5422 non-null   float64       
 6   atemp       5422 non-null   float64       
 7   humidity    5422 non-null   int64         
 8   windspeed   5422 non-null   float64       
 9   count       5422 non-null   int64         
 10  year        5422 non-null   int32         
 11  month       5422 non-null   int32         
 12  hour        5422 non-null   int32         
 13  weekday     5422 non-null   int32         
dtypes: datetime64[ns](1), float64(3), int32(4), int64(6)
memory usage: 508.4 KB


In [12]:
drop_features = 'datetime'
train_df = train_df.drop(drop_features, axis=1)
#train_df.to_csv(os.path.join(DATA_PATH, "bike_sharing_train_22.csv"))